In [ ]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.utils import shuffle

from tensorflow.keras.layers import Dense, RNN, LSTM, Flatten, TimeDistributed, LSTMCell, GlobalAveragePooling2D, Input, Dropout, Attention
from tensorflow.keras.layers import RepeatVector, Conv2D, SimpleRNN, GRU, Reshape, ConvLSTM2D, Conv2DTranspose, Conv3DTranspose

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
file_inp = '/content/gdrive/MyDrive/ADL_project/Data/set_2/input_res_4params_concatenated.npy'
file_tar = '/content/gdrive/MyDrive/ADL_project/Data/set_2/target_res_4params_concatenated.npy'

train_input = np.load(file_inp)
train_target = np.load(file_tar)

train_input, train_target = shuffle(train_input, train_target, random_state=0)

In [ ]:
#standardizing the data

mu_flux = train_input[:, :, 0].mean()
std_flux = train_input[:, :, 0].std()
mu_area = train_input[:, :, 1].mean()
std_area = train_input[:, :, 1].std()
train_input[:, :, 0] = (train_input[:, :, 0] - mu_flux) / std_flux
train_input[:, :, 1] = (train_input[:, :, 1] - mu_area) / std_area
train_target[:, :, 0] = (train_target[:, :, 0] - mu_flux) / std_flux
train_target[:, :, 1] = (train_target[:, :, 1] - mu_area) / std_area

In [ ]:
model = tf.keras.Sequential()
model.add(LSTM(256, input_shape=(33, 2), return_sequences = True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(RepeatVector(8))
model.add(LSTM(128, return_sequences=True))
model.add(TimeDistributed(Dense(2, activation = 'softplus')))
model.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_input[:30000, :, :2], train_target[:30000, :, :2], epochs = 400, batch_size = 256, validation_data = (train_input[30000:, :, :2], train_target[30000:, :, :2]))

In [ ]:
pred = model.predict(train_input[:, :, :2])

In [ ]:
#plot a prediction

rc_font = {"font.family" : "serif", 
      "mathtext.fontset" : "stix"}
plt.rcParams.update(rc_font)
plt.rcParams["legend.labelspacing"] = 0.001
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]




x = np.arange(0, 125, 3)
i = 30018   #this is for the id, choose an id above 30000 if you want to a prediction on the validation set
j = 0       #0 for mag flux plot, 1 for AR area plot
save = False
#y_true = list(train_input[i, :, j]) + list(train_target[i, :25, j])
y_true = list(train_input[i, :, j]) 
y_true_tar = [train_input[i, -1, j]] + list(train_target[i, :25, j])
y_pred = list(train_input[i, :, j]) + list(pred[i, :25, j])
fig, ax = plt.subplots(1, 1, figsize = (7, 7))
ax.set_title('Set {}'.format(i))

if j == 0:
  ax.plot(x[:33], np.array(y_true) *std_flux + mu_flux, c = 'black', label = 'Source')
  ax.plot(x[32:-1], np.array(y_true_tar) *std_flux + mu_flux, '--', c = 'black', label = 'Target' )
  ax.plot(x[32:-1], np.array(y_pred[32:]) *std_flux + mu_flux, '--', c = 'red', label = 'Prediction')
  ax.set_ylabel('Magnetic Flux (Mx)', fontsize = 15)
  #plt.savefig('Pred_Flux_{}.pdf'.format(i))
if j == 1:
  ax.plot(x[:33], np.array(y_true) *std_area + mu_area, c = 'black', label = 'Source' )
  ax.plot(x[32:-1], np.array(y_true_tar) *std_area + mu_area, '--', c = 'black', label = 'Target' )
  ax.plot(x[32:-1], np.array(y_pred[32:]) *std_area + mu_area, '--', c = 'red', label = 'Prediction')
  ax.set_ylabel('AR Area', fontsize = 15)
  #plt.savefig('Pred_Area_{}.pdf'.format(i))

ax.legend(frameon = False, fontsize = 15)
ax.set_xlabel(r'Time ($\times 12$min)', fontsize = 15)

ax.minorticks_on()
ax.tick_params(which = 'both', bottom = True, top = True, left = True, right = True)
ax.tick_params(which = 'major', length = 8, direction = 'in', labelsize = 12)
ax.tick_params(which = 'minor', length = 4, direction = 'in', labelsize = 12)
if j == 0 & save:
  #plt.savefig('Pred_Flux_{}.pdf'.format(i))
  pass
else:
  #plt.savefig('Pred_Area_{}.pdf'.format(i))
  pass

plt.show()